In [1]:
#Se inicia creando la base de datos EstudiantesUC siguiendo el esquema indicado.
import sqlite3
conn = sqlite3.connect('EstudiantesUC.db')
c = conn.cursor()

#Se borran las tablas si ya existen
c.execute('''Drop table if exists Curso''')
c.execute('''Drop table if exists Estudiante''')
c.execute('''Drop table if exists Estudiante_has_Curso''')
c.execute('''Drop table if exists EstudianteErasmus''')
c.execute('''Drop table if exists Profesor''')
c.execute('''Drop table if exists Profesor_has_Curso''')
# Creamos las tablas indicadas en el esquema, con sus claves primarias y foráneas, y las guardamos en la base de datos de UC
c.execute('''CREATE TABLE Curso
             (idCurso integer PRIMARY KEY, titulo VARCHAR(45) not NULL, creditos integer not NULL);''')
c.execute('''CREATE TABLE Estudiante
             (idEstudiante integer PRIMARY KEY, nombre VARCHAR(25) not NULL, apellidos VARCHAR(85) not NULL,
           numMatricula VARCHAR(15), direccion VARCHAR(100), telefono VARCHAR(15), email VARCHAR(45));''')
c.execute('''CREATE TABLE Estudiante_has_Curso
             (Estudiante_idEstudiante integer, Curso_idCurso integer,
          PRIMARY KEY (Estudiante_idEstudiante, Curso_idCurso),
          FOREIGN KEY (Estudiante_idEstudiante) REFERENCES Estudiante(idEstudiante),
          FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso));''')
#En EstudianteErasmus, como según el esquema la PRIMARY KEY la componen idEstudianteErasmus e idEstudiante, y idEstudiante es clave primaria en Estudiante, se añade UNIQUE a idEstudianteErasmus e idEstudiante para que no se repitan valores en la tabla
c.execute('''CREATE TABLE EstudianteErasmus
             (idEstudianteErasmus integer UNIQUE, idEstudiante integer UNIQUE, universidadOrigen VARCHAR(45),
          PRIMARY KEY (idEstudianteErasmus, idEstudiante));''')
c.execute('''CREATE TABLE Profesor
             (idProfesor integer PRIMARY KEY, nombre VARCHAR(25) not NULL, apellidos VARCHAR(85) not NULL,
          email VARCHAR(45));''')
c.execute('''CREATE TABLE Profesor_has_Curso
             (Profesor_idProfesor integer, Curso_idCurso integer,
          PRIMARY KEY (Profesor_idProfesor, Curso_idCurso),
          FOREIGN KEY (Profesor_idProfesor) REFERENCES Profesor(idProfesor),
          FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso));''')
conn.commit()
c.close()

In [2]:
#Se inicia creando la base de datos EstudiantesUoE siguiendo el esquema indicado.
import sqlite3
conn = sqlite3.connect('EstudiantesUoE.db')
c = conn.cursor()

#Se borran las tablas si ya existen
c.execute('''Drop table if exists Program''')
c.execute('''Drop table if exists SportActivity''')
c.execute('''Drop table if exists Student''')
c.execute('''Drop table if exists Subject''')
c.execute('''Drop table if exists StudentMarks''')
# Creamos las tablas indicadas en el esquema, con sus claves primarias y foráneas, y las guardamos en la base de datos de UoE
c.execute('''CREATE TABLE Program
             (idProgram integer PRIMARY KEY, name VARCHAR(45) not NULL, ueFees integer,
          overseasFees VARCHAR(45));''')
c.execute('''CREATE TABLE SportActivity
             (idSportActivity integer PRIMARY KEY, name VARCHAR(25) not NULL, durationWeeks integer not NULL,
           price integer not NULL);''')
#En la tabla Student se ha añadido un UNIQUE a Program_idProgram para que no pueda haber varios estudiantes con el mismo programa, al ser un uno a uno
c.execute('''CREATE TABLE Student
             (idStudent integer PRIMARY KEY, name VARCHAR(155) not NULL, dateOfBirth date not NULL,
          nationality VARCHAR(45), isErasmus boolean, homeUniversity VARCHAR(45), hasScolarship boolean,
          Program_idProgram integer UNIQUE, englishLevel CHAR(2), SportActivity_idSportActivity integer,
          FOREIGN KEY (Program_idProgram) REFERENCES Program(idProgram),
          FOREIGN KEY (SportActivity_idSportActivity) REFERENCES SportActivity(idSportActivity));''')
c.execute('''CREATE TABLE Subject
             (idSubject integer PRIMARY KEY, name VARCHAR(45) not NULL, year integer,
          Program_idProgram integer, FOREIGN KEY (Program_idProgram) REFERENCES Program(idProgram));''')
c.execute('''CREATE TABLE StudentMarks
             (Student_idStudent integer, Subject_idSubject integer, mark integer not NULL,
          PRIMARY KEY (Student_idStudent, Subject_idSubject));''')
conn.commit()
c.close()

In [3]:
#Se procede a rellenar la base de datos de UC con datos aleatorios.
from faker import Faker
import numpy as np
import random
numCursos = 4
numEstudiantes = 100
numEstudiantesErasmus = 20
numProfesores = 15
fake = Faker('es_ES')
conn = sqlite3.connect('EstudiantesUC.db')
c = conn.cursor()

for b in range(1,numCursos+1):
    idCurso = b
    titulo = fake.sentence()
    creditos = fake.random_int(min=40, max=60)
    c.execute('''INSERT INTO Curso(idCurso,titulo,creditos) VALUES (%s, "%s", "%s");''' % (idCurso, titulo, creditos));

for b in range(1,numEstudiantes+1):
    idEstudiante = b
    nombre = fake.name()
    apellidos = fake.last_name() + " " + fake.last_name()
    numMatricula = fake.random_int(min=1000, max=9999) 
    direccion = fake.address()
    telefono = fake.phone_number()
    email = fake.email()    
    c.execute('''INSERT INTO Estudiante (idEstudiante,nombre,apellidos,numMatricula,direccion,telefono,email) VALUES (%s, "%s", "%s", "%s", "%s", "%s", "%s");''' % (idEstudiante, nombre, apellidos, numMatricula, direccion, telefono, email));

for b in range(1,numEstudiantes+1):
    idEstudiante = b
    idCurso = fake.random_int(min=1, max=numCursos)
    c.execute('''INSERT INTO Estudiante_has_Curso(Estudiante_idEstudiante, Curso_idCurso) VALUES (%s, "%s");''' % (idEstudiante, idCurso));

vector = range(1,numEstudiantes+1)
listaEstudiantes = random.sample(vector, numEstudiantesErasmus)
for b in range(1,numEstudiantesErasmus+1):
    idEstudianteErasmus = b
    idEstudiante = listaEstudiantes[b-1]
    universidadOrigen = fake.company() #faker no proporciona universidades, así que he puesto empresas
    c.execute('''INSERT INTO EstudianteErasmus(idEstudianteErasmus, idEstudiante, universidadOrigen) VALUES (%s, "%s", "%s");''' % (idEstudianteErasmus, idEstudiante, universidadOrigen));

for b in range(1,numProfesores+1):
    idProfesor = b
    nombre = fake.name()
    apellidos = fake.last_name()
    email = fake.email()
    c.execute('''INSERT INTO Profesor(idProfesor,nombre,apellidos,email) VALUES (%s, "%s", "%s", "%s");''' % (idProfesor, nombre, apellidos, email));

for b in range(1, numCursos+1):
    idProfesor = fake.random_int(min=1, max=numProfesores)
    idCurso = b
    c.execute('''INSERT INTO Profesor_has_Curso(Profesor_idProfesor, Curso_idCurso) VALUES (%s, "%s");''' % (idProfesor, idCurso));

conn.commit()

c.close()

In [4]:
#Se procede a rellenar la base de datos de UoE con datos aleatorios.
from faker import Faker
import numpy as np
import random

numSportActivities = 9
numStudents = 100
numSubjects = numStudents*10
fake = Faker('en_GB')
conn = sqlite3.connect('EstudiantesUoE.db')
c = conn.cursor()

for b in range(1,numStudents+1):
    idProgram = b
    name = fake.name()
    ueFees = fake.random_int(min=70, max=1000)
    overseasFees = str(fake.random_int(min=3500, max=18000))
    c.execute('''INSERT INTO Program(idProgram,name,ueFees,overseasFees) VALUES (%s, "%s", "%s", "%s");''' % (idProgram, name, ueFees, overseasFees));

for b in range(1,numSportActivities+1):
    idSportActivity = b
    name = fake.name()
    durationWeeks = fake.random_int(min=1, max=32)
    price = fake.random_int(min=50, max=500) 
    c.execute('''INSERT INTO SportActivity(idSportActivity,name,durationWeeks,price) VALUES (%s, "%s", "%s", "%s");''' % (idSportActivity, name, durationWeeks, price));

for b in range(1, numStudents+1):
    idStudent = b
    name = fake.name()
    dateOfBirth = fake.date(pattern="%Y-%m-%d")
    nationality = fake.country()
    isErasmus = fake.boolean()
    homeUniversity = fake.company()
    hasScolarship = fake.boolean()
    idProgram = b
    englishLevel = fake.language_code()
    idSportActivity = fake.random_int(min=1, max=numSportActivities)
    c.execute('''INSERT INTO Student(idStudent,name,dateOfBirth,nationality,isErasmus,homeUniversity,hasScolarship,Program_idProgram,englishLevel,SportActivity_idSportActivity) VALUES (%s, "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s", "%s");''' % (idStudent, name, dateOfBirth, nationality, isErasmus, homeUniversity, hasScolarship, idProgram, englishLevel, idSportActivity));

v1 = range(1, numStudents+1)
a = np.array(random.choices(v1, k=numSubjects))
for b in range(1,numSubjects+1):
    idSubject = b
    name = fake.name()
    year = fake.year()
    idProgram = a[b-1]
    c.execute('''INSERT INTO Subject(idSubject,name,year,Program_idProgram) VALUES (%s, "%s", "%s", "%s");''' % (idSubject, name, year, idProgram));

for b in range(1,numSubjects+1):
    idStudent = a[b-1]
    idSubject = b
    mark = fake.random_int(min=0, max=10)
    c.execute('''INSERT INTO StudentMarks(Student_idStudent, Subject_idSubject, mark) VALUES (%s, "%s", "%s");''' % (idStudent, idSubject, mark));

conn.commit()

c.close()

In [5]:
#Se inicia creando la base de datos Estudiantes siguiendo el esquema indicado.
import sqlite3
conn = sqlite3.connect('Estudiantes.db')
c = conn.cursor()

#Se borran las tablas si ya existen
c.execute('''Drop table if exists Asignatura''')
c.execute('''Drop table if exists ActividadDeportiva''')
c.execute('''Drop table if exists Matricula''')
c.execute('''Drop table if exists Profesor''')
c.execute('''Drop table if exists Estudiante''')
c.execute('''Drop table if exists Profesor_has_Asignatura''')
c.execute('''Drop table if exists Nota_Estudiante_has_Asignatura''')
# Creamos las tablas indicadas en el esquema, con sus claves primarias y foráneas, y las guardamos en la base de datos de UC
c.execute('''CREATE TABLE Asignatura
             (idAsignatura integer PRIMARY KEY, nombre VARCHAR(45) not NULL, creditos integer, anho integer);''')
c.execute('''CREATE TABLE ActividadDeportiva
             (idActividadDeportiva integer PRIMARY KEY, nombre VARCHAR(45) not NULL, semanasDuracion integer not NULL,
           precio integer);''')
c.execute('''CREATE TABLE Matricula
             (idMatricula integer PRIMARY KEY, nombre VARCHAR(45) not NULL, tarifasUe integer,
           tarifasExtranjero VARCHAR(45));''')
c.execute('''CREATE TABLE Profesor
             (idProfesor integer PRIMARY KEY, nombre VARCHAR(25) not NULL, apellidos VARCHAR(85) not NULL,
          email VARCHAR(45));''')
c.execute('''CREATE TABLE Estudiante
             (idEstudiante integer PRIMARY KEY, nombreCompleto VARCHAR(155) not NULL, fechaNacimiento date,
          nacionalidad VARCHAR(45), direccion VARCHAR(100), telefono VARCHAR(15), email VARCHAR(45),
          esErasmus boolean, universidadOrigen VARCHAR(45), beca boolean, nivelIngles CHAR(2),
           Matricula_idMatricula integer UNIQUE, ActividadDeportiva_idActividadDeportiva integer,
            FOREIGN KEY (Matricula_idMatricula) REFERENCES Matricula(idMatricula),
            FOREIGN KEY (ActividadDeportiva_idActividadDeportiva) REFERENCES ActividadDeportiva(idActividadDeportiva));''')
c.execute('''CREATE TABLE Profesor_has_Asignatura
             (Profesor_idProfesor integer, Asignatura_idAsignatura,
          PRIMARY KEY (Profesor_idProfesor, Asignatura_idAsignatura),
          FOREIGN KEY (Profesor_idProfesor) REFERENCES Profesor(idProfesor),
          FOREIGN KEY (Asignatura_idAsignatura) REFERENCES Asignatura(idAsignatura));''')
c.execute('''CREATE TABLE Nota_Estudiante_has_Asignatura
             (Estudiante_idEstudiante integer, Asignatura_idAsignatura integer, nota integer,
          PRIMARY KEY (Estudiante_idEstudiante, Asignatura_idAsignatura),
          FOREIGN KEY (Estudiante_idEstudiante) REFERENCES Estudiante(idEstudiante),
          FOREIGN KEY (Asignatura_idAsignatura) REFERENCES Asignatura(idAsignatura));''')
conn.commit()
c.close()

In [6]:
#extriago los datos de la base de datos de estudiantes UC
conn = sqlite3.connect('EstudiantesUC.db')
c = conn.cursor()

#consulta sobre todo los datos de la tabla Curso
c.execute('''SELECT * FROM Curso''')
curUC = c.fetchall()

#consulta sobre todo los datos de la tabla Estudiante, uniendo la universidadOrigen de la tabla EstudianteErasmus
c.execute('''SELECT e.*, ee.universidadOrigen FROM Estudiante e LEFT JOIN EstudianteErasmus ee ON e.idEstudiante = ee.idEstudiante''')
estUC = c.fetchall()

#consulta sobre todo los datos de la tabla Profesor
c.execute('''SELECT * FROM Profesor''')
profUC = c.fetchall()

#consulta sobre todo los datos de la tabla Estudiante_has_Curso
c.execute('''SELECT * FROM Estudiante_has_Curso''')
estCurUC = c.fetchall()

#consulta sobre todo los datos de la tabla Profesor_has_Curso
c.execute('''SELECT * FROM Profesor_has_Curso''')
profCurUC = c.fetchall()

c.close()

In [7]:
#extriago los datos de la base de datos de estudiantes UoE
conn = sqlite3.connect('EstudiantesUoE.db')
c = conn.cursor()

#consulta sobre todo los datos de la tabla Program
c.execute('''SELECT * FROM Program''')
proUOE = c.fetchall()

#consulta sobre todo los datos de la tabla SportActivity
c.execute('''SELECT * FROM SportActivity''')
spoUOE = c.fetchall()

#consulta sobre todo los datos de la tabla Student
c.execute('''SELECT * FROM Student''')
stuUOE = c.fetchall()

#consulta sobre todo los datos de la tabla Subject
c.execute('''SELECT * FROM Subject''')
subUOE = c.fetchall()

#consulta sobre todo los datos de la tabla StudentMarks
c.execute('''SELECT * FROM StudentMarks''')
stumarkUOE = c.fetchall()

c.close()

In [8]:
#importamos datos estudiantes UC:
conn = sqlite3.connect('Estudiantes.db')
c = conn.cursor()

indAsignatura = 0
#insertamos informacion en la tabla Asignatura de la base de datos final
for b in range(0,len(curUC)):
    idAsignatura = curUC[b][0]
    nombre = curUC[b][1]
    creditos = curUC[b][2]
    anho = None
    c.execute('''INSERT INTO Asignatura(idAsignatura,nombre,creditos,anho) VALUES (?, ?, ?, ?);''' ,(idAsignatura, nombre, creditos, anho));
    indAsignatura+=1
conn.commit()

indMatricula = 0
#insertamos informacion en la tabla Matricula de la base de datos final
for b in range(0,len(estUC)):
    idMatricula = indMatricula + 1
    nombre = estUC[b][3]
    tarifasUe = None
    tarifasExtranjero = None
    c.execute('''INSERT INTO Matricula(idMatricula,nombre,tarifasUe,tarifasExtranjero) VALUES (?, ?, ?, ?);''' ,(idMatricula, nombre, tarifasUe, tarifasExtranjero));
    indMatricula+=1
conn.commit()

indProfesor = 0
#insertamos informacion en la tabla Profesor de la base de datos final
for b in range(0,len(profUC)):
    idProfesor = profUC[b][0]
    nombre = profUC[b][1]
    apellidos = profUC[b][2]
    email = profUC[b][3]
    c.execute('''INSERT INTO Profesor(idProfesor,nombre,apellidos,email) VALUES (?, ?, ?, ?);''' ,(idProfesor, nombre, apellidos, email));
    indProfesor+=1
conn.commit()

indEstudiante = 0
#insertamos informacion en la tabla Estudiante de la base de datos final
for b in range(0,len(estUC)):
    idEstudiante = estUC[b][0]
    nombreCompleto = estUC[b][1] + " " + estUC[b][2] 
    fechaNacimiento = None
    nacionalidad = None
    direccion = estUC[b][4]
    telefono = estUC[b][5]
    email = estUC[b][6]
    esErasmus = True if estUC[b][7] is not None else False
    universidadOrigen = estUC[b][7]
    beca = None
    nivelIngles = None
    Matricula_idMatricula = indEstudiante + 1
    ActividadDeportiva_idActividadDeportiva = None
    c.execute('''INSERT INTO Estudiante(idEstudiante,nombreCompleto,fechaNacimiento,nacionalidad,direccion,telefono,email,esErasmus,universidadOrigen,beca,nivelIngles,Matricula_idMatricula,ActividadDeportiva_idActividadDeportiva)
              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);''' ,(idEstudiante, nombreCompleto, fechaNacimiento, nacionalidad, direccion, telefono, email, esErasmus, universidadOrigen, beca, nivelIngles, Matricula_idMatricula, ActividadDeportiva_idActividadDeportiva));
    indEstudiante+=1
conn.commit()

#insertamos informacion en la tabla Profesor_has_Asignatura de la base de datos final
for b in range(0,len(profCurUC)):
    Profesor_idProfesor = profCurUC[b][0]
    Asignatura_idAsignatura = profCurUC[b][1]
    c.execute('''INSERT INTO Profesor_has_Asignatura(Profesor_idProfesor,Asignatura_idAsignatura) VALUES (?, ?);''' ,(Profesor_idProfesor, Asignatura_idAsignatura));
conn.commit()

#insertamos informacion en la tabla Nota_Estudiante_has_Asignatura de la base de datos final
for b in range(0,len(estCurUC)):
    Estudiante_idEstudiante = estCurUC[b][0]
    Asignatura_idAsignatura = estCurUC[b][1]
    nota = None
    c.execute('''INSERT INTO Nota_Estudiante_has_Asignatura(Estudiante_idEstudiante,Asignatura_idAsignatura,nota) VALUES (?, ?, ?);''' ,(Estudiante_idEstudiante, Asignatura_idAsignatura, nota));
conn.commit()

c.close()

Se ha utilizado los mismos índices que en la base de datos EstudiantesUC para conservar las relaciones entre entidades. Lo mismo hago con la de EstudiantesUoE.

In [9]:
##importamos datos estudiantes UoE:
conn = sqlite3.connect('Estudiantes.db')
c = conn.cursor()

#insertamos informacion en la tabla Asignatura de la base de datos final
for b in range(0,len(subUOE)):
    idAsignatura = indAsignatura + subUOE[b][0]
    nombre = subUOE[b][1]
    creditos = None
    anho = subUOE[b][2]
    c.execute('''INSERT INTO Asignatura(idAsignatura,nombre,creditos,anho) VALUES (?, ?, ?, ?);''' ,(idAsignatura, nombre, creditos, anho));
conn.commit()

#insertamos informacion en la tabla ActividadDeportiva de la base de datos final
for b in range(0,len(spoUOE)):
    idActividadDeportiva = spoUOE[b][0]
    nombre = spoUOE[b][1]
    semanasDuracion = spoUOE[b][2]
    precio = spoUOE[b][3]
    c.execute('''INSERT INTO ActividadDeportiva(idActividadDeportiva,nombre,semanasDuracion,precio) VALUES (?, ?, ?, ?);''' ,(idActividadDeportiva, nombre, semanasDuracion, precio));
conn.commit()

#insertamos informacion en la tabla Matricula de la base de datos final
for b in range(0,len(proUOE)):
    idMatricula = indMatricula + proUOE[b][0]
    nombre = proUOE[b][1]
    tarifasUe = proUOE[b][2]
    tarifasExtranjero = proUOE[b][3]
    c.execute('''INSERT INTO Matricula(idMatricula,nombre,tarifasUe,tarifasExtranjero) VALUES (?, ?, ?, ?);''' ,(idMatricula, nombre, tarifasUe, tarifasExtranjero));
conn.commit()

#insertamos informacion en la tabla Estudiante de la base de datos final
for b in range(0,len(stuUOE)):
    idEstudiante = indEstudiante + stuUOE[b][0]
    nombreCompleto = stuUOE[b][1]
    fechaNacimiento = stuUOE[b][2]
    nacionalidad = stuUOE[b][3]
    direccion = None
    telefono = None
    email = None
    esErasmus = stuUOE[b][4]
    universidadOrigen = stuUOE[b][5]
    beca = stuUOE[b][6]
    nivelIngles = stuUOE[b][8]
    Matricula_idMatricula = indMatricula + stuUOE[b][7]
    ActividadDeportiva_idActividadDeportiva = stuUOE[b][9]
    c.execute('''INSERT INTO Estudiante(idEstudiante,nombreCompleto,fechaNacimiento,nacionalidad,direccion,telefono,email,esErasmus,universidadOrigen,beca,nivelIngles,Matricula_idMatricula,ActividadDeportiva_idActividadDeportiva)
              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);''' ,(idEstudiante, nombreCompleto, fechaNacimiento, nacionalidad, direccion, telefono, email, esErasmus, universidadOrigen, beca, nivelIngles, Matricula_idMatricula, ActividadDeportiva_idActividadDeportiva));
conn.commit()

#insertamos informacion en la tabla Nota_Estudiante_has_Asignatura de la base de datos final
for b in range(0,len(stumarkUOE)):
    Estudiante_idEstudiante = indEstudiante + stumarkUOE[b][0]
    Asignatura_idAsignatura = indAsignatura + stumarkUOE[b][1]
    nota = stumarkUOE[b][2]
    c.execute('''INSERT INTO Nota_Estudiante_has_Asignatura(Estudiante_idEstudiante,Asignatura_idAsignatura,nota) VALUES (?, ?, ?);''' ,(Estudiante_idEstudiante, Asignatura_idAsignatura, nota));
conn.commit()

c.close()

In [10]:
#extraigo los datos de la base de datos de estudiantes
conn = sqlite3.connect('Estudiantes.db')
c = conn.cursor()

c.execute('''SELECT * FROM Estudiante''')
est = c.fetchall()

c.execute('''SELECT * FROM Asignatura''')
asig = c.fetchall()

c.execute('''SELECT * FROM Profesor''')
prof = c.fetchall()

c.execute('''SELECT * FROM Matricula''')
mat = c.fetchall()

c.execute('''SELECT * FROM ActividadDeportiva''')
act = c.fetchall()

c.execute('''SELECT * FROM Nota_Estudiante_has_Asignatura''')
estAsig = c.fetchall()

c.execute('''SELECT * FROM Profesor_has_Asignatura''')
profAsig = c.fetchall()

c.close()

In [11]:
print(spoUOE)
print(act)

print(profUC)
print(prof)

print(profCurUC)
print(profAsig)

print(estUC)
print(stuUOE)
print(est)

print([item[3] for item in estUC])
print(proUOE)
print(mat)

print(curUC)
print(subUOE)
print(asig)

print(estCurUC)
print(stumarkUOE)
print(estAsig)

[(1, 'Jill Reynolds', 2, 403), (2, 'Julie Allen-Morris', 11, 379), (3, 'Dr Toby Chapman', 1, 228), (4, 'Natasha Johnston', 29, 491), (5, 'Bryan Abbott', 3, 311), (6, 'Mr Tom Morris', 22, 93), (7, 'Dr Laura Lawson', 6, 268), (8, 'Ms Jenna George', 3, 478), (9, 'Dr Aaron Bolton', 15, 212)]
[(1, 'Jill Reynolds', 2, 403), (2, 'Julie Allen-Morris', 11, 379), (3, 'Dr Toby Chapman', 1, 228), (4, 'Natasha Johnston', 29, 491), (5, 'Bryan Abbott', 3, 311), (6, 'Mr Tom Morris', 22, 93), (7, 'Dr Laura Lawson', 6, 268), (8, 'Ms Jenna George', 3, 478), (9, 'Dr Aaron Bolton', 15, 212)]
[(1, 'Cristian Figueras Almansa', 'Prats', 'kpalau@example.com'), (2, 'Gisela Busquets-Durán', 'Arana', 'granadosteodora@example.net'), (3, 'Eli Martín Priego', 'Villalonga', 'pcano@example.com'), (4, 'Almudena Palacio Vigil', 'Revilla', 'gjauregui@example.net'), (5, 'Maite Larrea Aroca', 'Girón', 'nuriasanjuan@example.org'), (6, 'Enrique Castell Morcillo', 'Pelayo', 'wbuendia@example.com'), (7, 'Baudelio Araujo', 'Ber

He utilizado esta estructura de base de datos de forma que pudiese almacenar todos los datos de ambas bases de datos iniciales. He interpretado que Curso y Subject son relativamente equivalentes, de forma que he creado la tabla Asignatura, mientras que la tabla Program la he utilizado para hacer la tabla Matricula, al tener en Estudiante también el número de matrícula. En este caso no se ha unido con la tabla Asignatura para simplificarlo.
He unido la tabla Estudiante y EstudianteErasmus para obtener si el estudiante es de ERASMUS, quedando de igual forma que em la tabla Student.
En cuanto a la tabla Profesor, sus datos sólo se encuentran en la BBDD de EstudiantesUC, de igual forma ocurre con Profesor_has_Curso que ha pasado a ser Profesor_has_Asignatura.
Para la tabla ActividadDeportiva se actúa igual que con Profesor, pero estos datos se sacan de SportActivity de la BBDD EstudiantesUoE.
Con la lógica que se ha seguido, las tablas intermedias Estudiante_has_Curso y StudentMarks son directamente equivalentes, creandose Nota_Estudiante_has_Asignatura.

Me ha surgido algún problema para insertar datos en StudentMarks, puesto que Program se relaciona uno a uno con Student, y Program tiene una lista de Subjects, con lo cual cada Subject pertenece únicamente a un Student. Por lo tanto, si se quiere introducir más de un StudentMarks para cada Subject, debería repetirse el mismo idStudent e idSubject, y eso me contradecía la primary key doble en StudentMarks.

La estructura creada no es la más idónea pero pienso que resuelve en gran medida la unión de ambas bases de datos, a pesar de que Subject y Curso no sean exactamente lo mismo. Además, se han insertado los datos correctamente.
